# CIFAR-10 Image Classification - Training your own Convolutional Neural Network #
## EECS 16ML, Fall 2021 ##

Written by Richard Shuai, Dohyun Cheon, Larry Yan, Tony Shara, Adi Ganapathi. 

richardshuai@berkeley.edu, dohyuncheon@berkeley.edu, yanlarry@berkeley.edu, anthony.shara@berkeley.edu, avganapathi@berkeley.edu, 

### Table of contents ###
* [Introduction](#introduction)
* [Part 1: Introduction to Convolutions](#part1)  
◦ Implement the convolution operation  
◦ Visualize convolutions with different kernels
* [Part 2: Pooling](#part2)  
◦ Implement pooling operations  
◦ Understand the benefits of pooling  
* [Part 3: CIFAR-10 Image Classification](#part3)  
◦ Preprocess the dataset  
◦ Visualizing filters and activations from a basic CNN  
◦ Train your own model and plot learning curves
* [Part 4: Tricking a CNN Classifier with Adversarial Examples](#part4)  
◦ Generate an adversarial example  
◦ Classification of the adversarial example
* [Part 5: Implementing a Paper: ResNet](#part5)  
◦ Implement a residual block  
◦ Train a model using your residual block implementation
* [Conclusion and References](#conclusion)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from PIL import Image

from common_utils import *
import matplotlib.pyplot as plt

<a id="introduction"></a>
# Introduction

In this notebook, you will gain more familiarity with intuition about convolutional neural networks. You are also going to learn to classify images from the CIFAR-10 dataset with a simple convolutional neural network.
* First, we will review what a convolution by having you implement a convolution operation. We will then use this implementation to visualize convolutions with different kernels. 
* Next, you will learn about pooling layers and why they are necessary in convolutional neural networks.
* Then, we will visualize the filters and activations in a basic CNN in order to gain an intuition about what a convolutional neural network is actually learning.
* You will then implement your own convolutional neural network for image classification.
* With this model, you will learn to generate adversarial examples to feed into your newly trained model.
* Finally, you will be required to read the ResNet paper and implement a residual block layer.

Note that training the models in this notebook may take awhile, so make sure to start early.

<a id="part1"></a>
# Part 1: Introduction to Convolutions

Convolutional neural networks rely on convolutional layers in order to extract features from images for classification. First, we will introduce the idea of convolutions and how they transform an image. 
On a 2D input image $I$ and using a kernel $G$ with width $w$ and height $h$, we can perform a discrete 2D-convolution that can be mathematically represented as:

$$
(I ∗ G)[x, y] = \sum_{k=1}^{w} \sum_{l=1}^{h} I[x + k, y + l]G[k, l]
$$
  
Visually, the convolution operation looks like this: ([source](https://mlnotebook.github.io/post/CNN1/))
<img src="./visuals/convSobel.gif" width=500 align="center"/>

Note that the convolution output dimensions depends on the dimensions of both the image and the kernel.    

In convolutional neural networks, the input image often has multiple channels, so our kernel will have multiple channels to match the input. We sum across the channels so that the convolution operation becomes:
$$
(I ∗ G)[x, y] = \sum_c \sum_{k=1}^{w} \sum_{l=1}^{h} I[x + k, y + l, c]G[k, l, c]
$$

Visually, the convolution will look like this: ([source](https://towardsdatascience.com/types-of-convolution-kernels-simplified-f040cb307c37))
<img src="./visuals/conv_multiple_channels.png" width=300 height=300/>

## Implement the convolution operation

In this section, we will demonstrate what different convolutional filters look like. In order to do so, you will need fill in the code below to implement a naive version of the convolution operation with stride and padding. Do not worry about efficiency for now.

In [ ]:
def conv2d(X, kernel, stride=1, padding=(0, 0)):
    """
    Inputs:
     - X: Input image. Numpy array of shape (H, W, C).
     - kernel: Kernel. Numpy array of shape (H_k, W_k, C).
     - stride: Stride in both the y- and x-directions. Positive integer.
     - padding: Symmetric zero-padding specification for both the x- and y-directions. Tuple where padding[0]
                 and padding[1] specify padding in the y-direction and x-direction respectively.
    """
    # Expand for single channel dimension
    if X.ndim == 2:
        X = X[..., None]
    if kernel.ndim == 2:
        kernel = kernel[..., None]
    
    assert len(X.shape) == 3, 'Expected X to have 3 dimensions, but got shape {}'.format(X.shape)
    assert len(kernel.shape) == 3, 'Expected kernel to have 3 dimensions, but got shape {}'.format(kernel.shape)
    assert len(padding) == 2, 'Expected padding to be length 2, but got length {}'.format(len(padding))
    
    
    H, W, C = X.shape
    H_k, W_k, _ = kernel.shape
    pad_y, pad_x = padding
    
    out = None
    ### BEGIN CODE ###

    ### END CODE ###
    
    return out

In [ ]:
# Sanity checks for conv2d implementation
X = np.random.randint(0, 50, size=(7, 7, 1)).astype(np.float32)
K = np.random.randint(0, 5, size=(3, 3, 1)).astype(np.float32)

# With stride=1, padding=(0, 0)
conv_naive = conv2d(X, K)
conv_tf = tf.nn.conv2d(X[None], K[..., None], strides=1, padding='VALID').numpy().squeeze()
assert np.all(conv_naive == conv_tf)

# With stride=2, padding=(0, 0)
conv_naive = conv2d(X, K, stride=2)
conv_tf = tf.nn.conv2d(X[None], K[..., None], strides=2, padding='VALID').numpy().squeeze()
assert np.all(conv_naive == conv_tf)

# With stride=2, padding=(2, 3)
conv_naive = conv2d(X, K, stride=2, padding=(2, 3))
conv_tf = tf.nn.conv2d(X[None], K[..., None], strides=2, padding=((0, 0), (2, 2), (3, 3), (0, 0))).numpy().squeeze()
assert np.all(conv_naive == conv_tf)

## Visualize convolutions with different kernels 

### Kernels for Edge Detection

Now that we have defined the convolution operation, we will use this to visualize the effects of convolution with certain kernels. Here, we visualize the effects of convolving 4 different filters on the cameraman image:  

<img src="./visuals/camera_man.png" width=200 align="center"/>

The kernels that we will be visualizing are listed below.  
Identity Kernel:
$
\begin{bmatrix} 
0 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 0 
\end{bmatrix}
$  
Edge detection kernel 1:
$
\begin{bmatrix} 
-1 & -1 & -1 \\
0 & 0 & 0 \\
1 & 1 & 1 
\end{bmatrix}
$  
Edge detection kernel 2:
$
\begin{bmatrix} 
-1 & 0 & 1 \\
-1 & 0 & 1 \\
-1 & 0 & 1 
\end{bmatrix}
$  
Edge detection kernel 3:
$
\begin{bmatrix} 
-1 & -1 & -1 \\
-1 & 8 & -1 \\
-1 & -1 & -1 
\end{bmatrix}
$  

In [ ]:
img = np.asarray(Image.open('./visuals/camera_man.png'))

kernels = np.array([[[0, 0, 0],
                    [0, 1, 0],
                    [0, 0, 0]],
                    
                [[-1, -1, -1],
                [0, 0, 0],
                [1, 1, 1]],
                
                [[-1, 0, 1],
                [-1, 0, 1],
                [-1, 0, 1]],
                
                [[-1, -1, -1],
                [-1, 8, -1],
                [-1, -1, -1]]
])

In [ ]:
convolved_imgs = []
labels = ['Identity kernel', 'Edge detection 1', 'Edge detection 2', 'Edge detection 3']

for i, kernel in enumerate(kernels):
    convolved_img = conv2d(img, kernel)
    convolved_imgs.append(convolved_img)
    
plot_labeled_image_grid(np.array(convolved_imgs), labels, cmap='gray', sizes=4, n_cols=4)

**Describe what you notice about the different convolution outputs. How do the edge detection kernels differ?**

**Your answer:**

### Gaussian blurring

As a quick demonstration, beyond just edge detection, convolving images with a kernel can also achieve a blurring effect. Here, we apply a Gaussian blur by convolving a certain matrix over the image. This matrix is based on the probability density function for a 2D Gaussian distribution with mean $0$ and standard deviation $\sigma$:  

$$ f(x, y) = \frac{1}{2\pi \sigma^2}e^{-\frac{x^2 + y^2}{2\sigma^2}}$$.  
A kernel is obtained from this density function by integrating over each pixel. Doing this with $\sigma=1$ for a  5x5 yields this discrete approximation:  

$$ \frac{1}{256} 
\begin{bmatrix} 
1 & 4 & 6 & 4 & 1 \\
4 & 16 & 24 & 16 & 4 \\ 
6 & 24 & 36 & 24 & 6 \\
4 & 16 & 24 & 16 & 4 \\
1 & 4 & 6 & 4 & 1
\end{bmatrix}
$$


In [ ]:
gaussian_kernel = np.array([[1, 4, 6, 4, 1],
                            [4, 16, 24, 16, 4], 
                            [6, 24, 36, 24, 6],
                            [4, 16, 24, 16, 4],
                            [1, 4, 6, 4, 1]]) / 256

# Blurred image
blurred_img = conv2d(img, gaussian_kernel)

# Plot images side by side
fig = plt.figure(figsize=(10, 5))
s1 = fig.add_subplot(1, 2, 1)
s1.set_title(r'Original Image')
plt.imshow(img, cmap='gray')

s2 = fig.add_subplot(1, 2, 2)
s2.set_title(r'Gaussian Blur with $\sigma=1$')
plt.imshow(blurred_img, cmap='gray')
plt.show()

<a id="part2"></a>
# Part 2: Pooling

Pooling layers are also an essential component of convolutional neural networks, both for parameter efficiency and for extracting information from larger contexts. Pooling reduces the size of an image or feature map by performing a function over slices of an image. The most commonly used type of pooling is **max pooling**, in which the maximum element is taken over the size of the filter. For example, it is common to use a max pooling with a filter of size 2x2 and stride of 2. This operation is depicted below ([source](https://computersciencewiki.org/index.php/Max-pooling_/_Pooling)):

<img src="./visuals/MaxpoolSample2.png" width=400 align="center"/>

In the following section, you will be required to implement some of the types of pooling operation before using your function to visualize how pooling transforms an image.

## Implement Pooling Operations

Fill in the code below to implement a pooling operation on an image. You will be required to implement both max pooling and average pooling, in average pooling, you simply take the average over the elements in the filter instead of the maximum.

In [ ]:
def pool2d(X, pool_size, stride, pool_type='max'):
    """
    Inputs:
     - X: Input image. Numpy array of shape (H, W, C).
     - pool_size: Size of the pooling filter in both the y- and x- directions. Positive integer.
     - stride: Stride in both the y- and x-directions. Positive integer.
     - pool_type: either 'max' or 'average' to denote pool type. String.
    """
    pool_types = {'max', 'average'}
    # Expand for single channel dimension
    if X.ndim == 2:
        X = X[..., None]
    
    assert len(X.shape) == 3, 'Expected X to have 3 dimensions, but got shape {}'.format(X.shape)
    assert pool_type in pool_types, 'Pool type not implemented: {}'.format(pool_type)
    
    H, W, C = X.shape
    
    out = None
    ### BEGIN CODE ###

    ### END CODE ###
    return out

In [ ]:
# Sanity checks for pool2d implementation
X = np.random.randint(0, 20, size=(7, 7, 3)).astype(np.float32)

# With pool_size=2, stride=2, pool_type='max'
pool_naive = pool2d(X, pool_size=2, stride=2, pool_type='max')
pool_tf = tf.nn.max_pool2d(X[None], ksize=2, strides=2, padding='VALID').numpy()
assert np.all(pool_naive == pool_tf)

# With pool_size=2, stride=2, pool_type='average'
pool_naive = pool2d(X, pool_size=2, stride=2, pool_type='average')
pool_tf = tf.nn.avg_pool2d(X[None], ksize=2, strides=2, padding='VALID').numpy()
assert np.all(pool_naive == pool_tf)

## Understand the Benefits of Pooling

Pooling with filter size 2x2 and a stride of 2 reduces the number of pixels in an image by a factor of 4 while keeping the main features of the image. This provides several benefits:
* By reducing the size of the image, successive convolution operations are not as expensive. This is especially important in deeper networks, where the number of channels of each feature map is increased in order to extract more features from an image. 
* By downsampling the image, convolutions applied over the downsampled image can have a larger receptive field. In other words, they can obtain context from a larger area.
* Because max pooling extracts the dominant value in each region, it can extract sharp features such as edges and other important features. Max pooling has empirically been shown to improve results in many CNN architectures.

As you can see below, max pooling can be necessary to keep feature maps to a reasonable size, especially with a large number of channels ([source](https://computersciencewiki.org/index.php/Max-pooling_/_Pooling)).
<img src="./visuals/MaxpoolSample.png" width=400 align="center"/>

In this section, we will use your pooling function to visualize the outcomes of repeatedly applying pooling on both a greyscale of a boat and color image of fruits.  

<table><tr>
<td> <img src="./visuals/boat.png" width=200/> </td>
<td> <img src="./visuals/fruits.png" width=200/> </td>
</tr></table>

In [ ]:
# Load in boat and fruits images.
boat_img = np.asarray(Image.open('./visuals/boat.png'))
fruits_img = np.asarray(Image.open('./visuals/fruits.png'))

### Greyscale image: boats

Here, we repeatedly apply pooling to the greyscale image with a 2x2 filter size and a stride of 2. We plot the outputs from average and max pooling at each step. We do the same with the fruits image below.

In [ ]:
num_pools = 5

fig=plt.figure(figsize=(5 * num_pools, 10))
rows, columns = 2, num_pools

avg_pooled_img = boat_img
max_pooled_img = boat_img

for i in range(num_pools):
    # Plot max pool
    s = fig.add_subplot(rows, columns, i+1)
    s.set_title("Max pool #{}".format(i+1))
    max_pooled_img = pool2d(max_pooled_img, pool_size=2, stride=2, pool_type='max')
    plt.imshow(max_pooled_img, cmap='gray')
    
    # Plot ground truth
    s = fig.add_subplot(rows, columns, i+columns+1)
    s.set_title("Average pool #{}".format(i+1))
    avg_pooled_img = pool2d(avg_pooled_img, pool_size=2, stride=2, pool_type='average')

    plt.imshow(avg_pooled_img, cmap='gray')

### Color image: Fruits

In [ ]:
fig=plt.figure(figsize=(5 * num_pools, 10))
rows, columns = 2, num_pools

avg_pooled_img = fruits_img
max_pooled_img = fruits_img

for i in range(num_pools):
    # Plot max pool
    s = fig.add_subplot(rows, columns, i+1)
    s.set_title("Max pool #{}".format(i+1))
    max_pooled_img = pool2d(max_pooled_img, pool_size=2, stride=2, pool_type='max')
    plt.imshow(normalize(max_pooled_img))
    
    # Plot ground truth
    s = fig.add_subplot(rows, columns, i+columns+1)
    s.set_title("Average pool #{}".format(i+1))
    avg_pooled_img = pool2d(avg_pooled_img, pool_size=2, stride=2, pool_type='average')

    plt.imshow(normalize(avg_pooled_img))

**Describe the differences you notice between max pooling and average pooling. Looking at the scales of the images, can you see why pooling is helpful for increasing the receptive field of convolutions?**

**Your answer:**

<a id="part3"></a>

# Part 3: CIFAR-10 Image Classification

Now that we have a better understanding of how convolutions and pooling work and how they transform images, we will learn how to use a convolutional neural network to extract features from an image and perform image classification on CIFAR-10.

## Preprocess the dataset

In [ ]:
# Load in the CIFAR-10 Dataset
(X, Y), (X_test, Y_test) = tf.keras.datasets.cifar10.load_data()

### Visualize a few images from the dataset

First, we'll plot some of the images to understand what we're actually classifying. The possible classes are given in the label_names array. Feel free to change num_visualize to see more images.

In [ ]:
label_names = np.array(['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'])

# Input the number of images to visualize
num_visualize = 20

# Construct a grid to plot images
labels = label_names[Y[:num_visualize].squeeze()] # Get label names in English from their 0-9 encodings
plot_labeled_image_grid(X[:num_visualize], labels, title='CIFAR 10 Images')

Note that the images from the CIFAR-10 dataset are unfortunately low resolution because each image is only 32x32 pixels. These images are small to help us focus on the classification task without worrying too much about computing power. 

### Create Train, Validation, and Test Datasets

Fill in the code to split the dataset into train and validation sets. Here, we will use a 90:10 split to create our train and validation datasets.

In [ ]:
TRAIN_SPLIT = 0.9
NUM_TRAIN_SAMPLES = int(TRAIN_SPLIT * X.shape[0])

In [ ]:
def train_val_split(X, Y, num_train_samples):
    N = X.shape[0]

    ### BEGIN CODE ###

    ### END CODE ###

    return X_train, Y_train, X_val, Y_val

In [ ]:
# Split dataset with our function
X_train, Y_train, X_val, Y_val = train_val_split(X, Y, NUM_TRAIN_SAMPLES)

In [ ]:
# Sanity check for shapes
assert(X_train.shape == (45000, 32, 32, 3))
assert(X_val.shape == (5000, 32, 32, 3))
assert(Y_train.shape == (45000, 1))
assert(Y_val.shape == (5000, 1))

### Image Preprocessing

Preprocess the images in our dataset. There are several ways you can do this. We have provided some examples below.
* Normalize the data from 0 to 1. 
* Subtract the mean of the training data
* Z-score standardization
* Dividing by 255 (for images ranging from 0 to 255)

Remember to apply the same preprocessing to the test and validation datasets as you do the train datasets. (For example, if the mean is calculated across the training data, subtract that same value from the validation and test sets).

In [ ]:
X_train = X_train.astype(np.float32)
X_val = X_val.astype(np.float32)
X_test = X_test.astype(np.float32)

### BEGIN CODE ###

### END CODE ###

## Visualizing filters and activations from a basic CNN

In this section, we define and train a basic CNN and visualize the learned filters from the first layer as well as layer activations when classifying images.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=9, padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(32, kernel_size=3, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(32, kernel_size=3, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Activation('softmax')
])

For this classification task, we use categorical cross entropy loss. In Keras, since our data is not one-hot encoded, we use sparse categorical cross entropy loss, but this is equivalent to categorical cross entropy loss for one-hot encoded labels. This loss for each sample is defined as: 
$$L(x) = \displaystyle\sum_{k=1}^C -y_{k} \cdot log(\sigma(s)_k)$$

* $y_k = 1$ if the sample belongs to class $k$ and $y_k = 0$ is it does not belong to class $k$.
* $s$ is the score obtained from the last layer of the network before the activation function.
* $\sigma$ is the softmax activation function:
$$\sigma(z)_k = \frac{e^{z_k}}{\sum_{j=1}^{C} e^{z_j}}$$  

The loss is therefore
$$Loss = \frac{1}{N} \displaystyle\sum_{i=1}^{N} \displaystyle\sum_{k=1}^C -y_{i, k} \cdot log(\sigma(s_i)_k) $$  

where we average over the batch with batch size $N$.  

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

We now compile and view a summary of our model. We can observe that most of the weights in our model come from the fully connected layers at the end, and that the pooling layers do not require any parameters.

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=10)

### Visualize learned filters

With this model, we can visualize the filters from the first convolutional layer to see what features the filters are detecting. As you should see, some of these filters look like edge-detecting filters.   
Note that here, we only extract the filters from the first convolutional layer because it is generally more difficult to interpret the convolutional filters from deeper layers. This is because these deeper layers act on encoded representations of images.

In [ ]:
# Get the layers of the model in a list
layers = model.layers

# Extract the filters from the first convolutional layer
layer_0 = layers[0]
filters, biases = layer_0.get_weights()
filters = normalize(filters)

plot_image_grid(filters.transpose(3, 0, 1, 2), title='Learned Convolutional Filters')

**Do any of these filters appear meaningful or recognizable?** *(Hint: you may recognize some filters from earlier in this notebook)*

**Your answer:**

### Visualize activations for certain images

We can also visualize the activations from the network when it makes predictions on certain images. This helps us understand how each filter encodes the image and transforms it through successive convolutional layers. Here, we demonstrate a visualization for the first layer activations. Fill in the code to visualize activations from the second and third convolutional layers. 

*(Hint: Inspect model.layers in order to understand the outputs of the activation_model.)*

In [ ]:
k = 2 # Change k to visualize different images in the training dataset.

img = X_train[k]
label = label_names[Y_train[k].squeeze()]

fig = plt.figure(figsize=(2, 2))
plt.imshow(img)
plt.title(label)
plt.show()

In [ ]:
layer_outputs = [layer.output for layer in model.layers]
activation_model = Model(model.input, layer_outputs)
activations = activation_model(img[np.newaxis, ...])

first_conv_activations = activations[0]

plot_image_grid(tf.squeeze(tf.transpose(first_conv_activations, [3, 0, 1, 2])), title='First Convolutional Layer Activations')

In [ ]:
second_conv_activations = None
third_conv_activations = None

### BEGIN CODE ###

### END CODE ###

plot_image_grid(tf.squeeze(tf.transpose(second_conv_activations, [3, 0, 1, 2])), title='Second Convolutional Layer Activations')
plot_image_grid(tf.squeeze(tf.transpose(third_conv_activations, [3, 0, 1, 2])), title='Third Convolutional Layer Activations')

**Describe what you notice about the activations. How is the image transformed over successive layers?**

**Your answer:**

## Train Your Own Model

Now that we've developed a better intuitive understanding of how convolutional neural networks learn, it's time for you to implement a better architecture that can achieve a higher validation loss. Train a model to achieve a validation loss of at least 70% on CIFAR-10. 

Some tricks that might be helpful:
* Common kernel sizes are 3x3 or 5x5. Remember that larger kernel sizes can require a much larger number of parameters.
* Batch normalization can vastly speed up training by rescaling the inputs to each layer. It can also be act to regularize your network. You can read more about it here: https://arxiv.org/abs/1502.03167
* Architectures commonly make use of a CONV -> BATCH_NORM -> ACTIVATION -> CONV -> BATCH_NORM -> ACTIVATION -> POOLING pattern.
* Other ideas to test out include changing hyperparameters such as stride, adding dropout layers, and different optimization algorithms (https://ruder.io/optimizing-gradient-descent/).

You are given a lot of freedom to write the code for your architecture. Feel free to change any starter code if you feel like it's necessary.

In [ ]:
model = None

### BEGIN CODE ###

## END CODE ###

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=10)

### Visualize the learning curves

Plot the train loss and validation loss as a function of the number of epochs trained.

*(Hint: the history returned from model.fit may be helpful here.)*

In [ ]:
### BEGIN CODE ###

### END CODE ###

### Report the final loss on our test set

We now evaluate the model on the test set to get the final test accuracy for the model. Note that we only evaluate on the test set once, at the end.

In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)
print('Final test set accuracy: {}'.format(accuracy))

<a id="part4"></a>

# Part 4: Tricking a CNN Classifier with Adversarial Examples

As we have seen, CNNs are capable of classifying images with high accuracy. However, it turns out that it is very easy to "trick" a trained CNN classifier with *adversarial examples*. By taking a correctly classified image and adding a small but directed perturbation to the image, we can force the CNN to misclassify the image with high confidence. In this section, we will briefly demonstrate how we can cause your newly trained model to misclassify images with methods based on this paper: https://arxiv.org/abs/1412.6572. 

The change that we will be making is as follows:
$$\vec{x}_{adversarial} = \vec{x} + \epsilon \cdot sign(\nabla_x J(\theta, \vec{x}, y))$$  
where $J(\theta, \vec{x}, y))$ is the loss resulting from evaluating the model on an input image $\vec{x}$ with model parameters $\theta$, and $\epsilon$ is a small constant. Notice that we are taking the gradient with respect to the input image $x$ this time. Intuitively, this means that we are changing the image by making a small step in the direction that maximizes the loss the most under a max-norm constraint.

Fill in the code below to implement this update. You will need to compute the gradient of the loss with respect to the model input and set it to grad_x.  
*(Hint: [tf.GradientTape()](https://www.tensorflow.org/api_docs/python/tf/GradientTape) may be helpful here)*

## Generate an adversarial example

In [ ]:
def generate_adversarial_example(model, x, y_true, eps=0.01):
    if len(x.shape) == 3:
        x = x[np.newaxis, ...]

    x = tf.convert_to_tensor(x, dtype=tf.float32)
    
    grad_x = None
    adversarial_example = None

    ### BEGIN CODE ###

    ### END CODE ### 
    
    return (grad_x, adversarial_example)

For the purpose of our example, we want to make sure we perturb an image that the model would ordinarily be able to correctly classify. Obtain the index of such a correctly classified image in X_train and set it to variable idx.

In [ ]:
idx = None

### BEGIN CODE ###

### END CODE ###

In [ ]:
# Visualize the original image and your model's prediction
img = X_train[idx]
y_true = Y_train[idx].squeeze()

plt.title(label_names[y_true])
plt.imshow(img)
plt.show()

With this correctly classified image, fill in the code below to generate an adversarial example that your model is likely to misclassify.

In [ ]:
eps = 0.01
grad_x = None
adversarial_img = None

### BEGIN CODE ###

### END CODE ###

## Classification of the adversarial example

Run the forward pass of the model to find the prediction it makes on the adversarial input and set the label to label_new. You may need to slightly tune $\epsilon$ from above or change the original image in order to misclassify the image.

In [ ]:
label_new = None

### BEGIN CODE ###

### END CODE ###

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))
ax1.set_title('Original prediction: {}'.format(label_names[y_true]), fontsize=16)
ax1.imshow(img)

ax2.set_title(r'$\vec{\epsilon} \cdot sign(\nabla_x J(\theta, \vec{x}, y))$', fontsize=16)
ax2.imshow(normalize(tf.squeeze(grad_x)))

ax3.set_title('Adversarial prediction: {}'.format(label_new), fontsize=16)
ax3.imshow(normalize(tf.squeeze(adversarial_img)))

plt.show()


You can see that the two images look identical to the human eye. This demonstrates that although CNNs appear to classify images in a very intuitive way by extracting recognizable features, the model can still fail to classify images that are perturbed in a certain way.

<a id="part5"></a>
# Part 5: Implementing a Paper: ResNet

### Implement a residual block

In developing machine learning models, it is valuable practice to be able to read a paper, understand it conceptually, and then to actually implement its architecture. Here, we will implement a key component of the architecture from the ResNet paper: residual blocks. 

Read through the ResNet paper, and then fill in the code to implement a residual block with two layers.  
The ResNet paper can be found here: https://arxiv.org/abs/1512.03385

Below, we've included a diagram from the picture for what specifically you should implement: use convolutional layers for the weight layers, and if the dimensions of $\vec{x}$ and $\mathcal{F}$ do not match, use a 1x1 convolution as the linear projection to match the dimensions.  


<img src="visuals/resnet_building_block.png" alt="Drawing" style="width: 50em;"/>

**What is the intuition behind using residual blocks in a neural network? Why are they useful?**

**Your answer:**

In [ ]:
class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(ResidualBlock, self).__init__()
        ### BEGIN CODE ###

        ### END CODE ###
        
    def call(self, x):
        ### BEGIN CODE ###

        ### END CODE ###
        return out

In [ ]:
### Sanity checking for the shapes of your outputs ###
x = tf.random.uniform([1, 5, 5, 3])

residual_block_1 = ResidualBlock(in_channels=3, out_channels=3, kernel_size=3)
residual_block_2 = ResidualBlock(in_channels=3, out_channels=7, kernel_size=3)

assert tuple(tf.shape(residual_block_1(x))) == (1, 5, 5, 3)
assert tuple(tf.shape(residual_block_2(x))) == (1, 5, 5, 7)

### Train a model using your residual block implementation

Here, you can test your residual block implementation by directly using it to train a model. As you implement your model, you should keep in mind that residual blocks tend to shine the most in deeper networks.    

Again, we have provided starter code for the loss function and fitting, but feel free to change the code if you need to. Achieve a validation accuracy of over 70%, but do not worry about outperforming the model you trained previously.

In [ ]:
model = None

### BEGIN CODE ###

## END CODE ###

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=10)

### Report your final loss on the test set

In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)
print('Final test set accuracy: {}'.format(accuracy))

<a id="conclusion"></a>
# Conclusion
Great job! You've implemented a convolutional layers and successfully built your own CNN to classify images, learning how you can visualize the filters and activations of a trained model along the way. You also saw a bit about how small but selective perturbations can cause your CNN to misclassify relatively easily. Finally, you were able to practice reading and implementing a machine learning research paper. 

# References
Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift (Sergey Ioffe, Christian Szegedy)  
https://arxiv.org/abs/1502.03167

An overview of gradient descent optimization algorithms (Sebastian Ruder)  
https://ruder.io/optimizing-gradient-descent/


Explaining and Harnessing Adversarial Examples (Ian J. Goodfellow, Jonathon Shlens, Christian Szegedy)  
https://arxiv.org/abs/1412.6572

Deep Residual Learning for Image Recognition (Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun)  
https://arxiv.org/abs/1512.03385